In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pykeen
import torch
from pykeen.pipeline import pipeline

In [2]:
dataset = 'Nations'
num_epochs = 5
embedding_dim = 10
lbda = 0.1
loss = 'MarginRankingLoss'

In [8]:
from pykeen.models import TransR

class ModifiedTransR(TransR):
    
    def score_hrt(self, hrt_batch: torch.LongTensor) -> torch.FloatTensor:  # noqa: D102
        # Get embeddings
        h = self.entity_embeddings(indices=hrt_batch[:, 0]).unsqueeze(dim=1)
        r = self.relation_embeddings(indices=hrt_batch[:, 1]).unsqueeze(dim=1)
        t = self.entity_embeddings(indices=hrt_batch[:, 2]).unsqueeze(dim=1)
        m_r = self.relation_projections(indices=hrt_batch[:, 1]).view(-1, self.embedding_dim, self.relation_dim)

        return self.interaction_function(h=h, r=r, t=t, m_r=m_r).view(-1, 1)

In [9]:
result2 = pipeline(
    model=ModifiedTransE,
    dataset=dataset,
    random_seed=1235,
    device='gpu',
    training_kwargs=dict(num_epochs=num_epochs),
    model_kwargs=dict(embedding_dim=embedding_dim),
    loss=loss
)
model2 = result2.model
model2

NameError: name 'rel_hnq' is not defined

In [ ]:
result2.plot_losses()
plt.show()

In [ ]:
comp_models = ['StructuredEmbedding','TransE','RotatE','HolE', 'TransR']
comp_results = []
for comp_model in comp_models:
    print('Running {}'.format(comp_model))
    result = pipeline(
        dataset=dataset,
        model=comp_model,
        random_seed=1235,
        device='gpu',
        training_kwargs=dict(num_epochs=num_epochs),  # Shouldn't take more than a minute or two on a nice computer
        model_kwargs=dict(embedding_dim=embedding_dim),
        loss=loss
    )
    comp_results.append(result)

In [ ]:
plt.plot(np.arange(len(result2.losses)),result2.losses,label='Sheaf SE')
for i in range(len(comp_models)):
    comp_model = comp_models[i]
    comp_result = comp_results[i]
    plt.plot(np.arange(len(comp_result.losses)),comp_result.losses,label=comp_model)
plt.ylabel(str(result.model.loss).replace('()',''))
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
res_df = result2.metric_results.to_df()

In [ ]:
compto = 1

In [ ]:
comp_results[compto].metric_results.to_df()

In [ ]:
res_df['diff'] = res_df.Value - comp_results[compto].metric_results.to_df().Value
res_df

In [ ]:
# comp_results[0].model.score_all_triples()

In [ ]:
# model2.score_all_triples()